In [26]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand
import time

In [80]:
#=========================================================================================


class fit():
    def __init__(self,epsilon = 0.7, learning_rate = 0.1, max_iter = 151,regularizer = 0.1):
        self.epsilon = epsilon
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.regularizer = regularizer
        
        
    def operators(configuration):
        #O = <S_i S_j>
        system_size,sample_size = configuration.shape
        Op = np.zeros((int(system_size*(system_size-1)/2), sample_size))

        jindex = 0
        for index in range(system_size-1):
            temp = configuration[index,:] * configuration[index+1:,:]
            Op[jindex: jindex + system_size - index -1 , :] = temp
            jindex += system_size - index - 1
        return Op
        
        
    def em1(self, ops):
        n_ops = ops.shape[0]
        np.random.seed(13)
        w = np.random.rand(n_ops)-0.5
        for i in range(self.max_iter):

            energies_w = w@ops

            energies_max = energies_w.max()  
            probs_w = np.exp((energies_w-energies_max)*(self.epsilon-1)) 
            z_data = np.sum(probs_w)
            probs_w /= z_data
            ops_expect_w = np.sum(probs_w[np.newaxis,:]*ops,axis=1)


            w += self.learning_rate*(ops_expect_w - w*self.epsilon - self.regularizer*w)

        return w
    
    def em2(self, ops):
        n_ops = ops.shape[0]
        np.random.seed(13)
        w = np.random.rand(n_ops)-0.5
        w = torch.from_numpy(w).cuda()
        ops = torch.from_numpy(ops).cuda()
        
        for i in range(self.max_iter):

            energies_w = w@ops

            probs_w = torch.exp((energies_w)*(self.epsilon-1)) 
            z_data = torch.sum(probs_w)
            probs_w /= z_data
            ops_expect_w = torch.sum(probs_w[:,np.newaxis]*ops,axis=1)


            w += self.learning_rate*(ops_expect_w - w*self.epsilon - self.regularizer*w)

        return w

In [81]:
EM = fit()
config = np.random.choice([-1.,1.] , size=(10,1000000), replace = True)
OPS = fit.operators(config)
W = EM.em1(OPS)

In [82]:
W

array([ 2.91825370e-04,  1.01377354e-03, -8.78593481e-04,  3.82116400e-05,
       -4.27138188e-04,  8.99150606e-05,  2.29238213e-04,  1.69632493e-03,
        2.07985728e-04,  8.26585363e-04, -6.69446152e-04,  2.29800990e-04,
        3.12484900e-04,  9.97834158e-05,  7.88988689e-04,  6.35765193e-04,
        9.45532262e-04, -8.75749165e-04,  4.92365758e-04,  4.88055260e-05,
       -7.27012753e-04,  8.33539922e-05,  1.53612683e-03, -1.57610934e-03,
        3.64080658e-04, -1.08089558e-03, -1.43714899e-03,  2.26849602e-03,
        1.67630282e-04, -1.25651594e-03,  6.68353214e-04,  1.64367690e-03,
        2.08488866e-03, -2.56947745e-04, -1.36492408e-04,  2.00233253e-04,
        1.43404432e-03, -1.38303488e-04,  8.07338813e-04,  2.29672919e-03,
       -4.30915248e-04, -9.39469311e-04,  1.48156992e-03,  2.66334275e-03,
       -2.67761682e-04])

In [65]:
config[:,0]

array([ 1.,  1., -1.])

In [66]:
print(OPS[:,0])

[ 1. -1. -1.]


In [ ]:
    
        
# convert s[n_seq,n_var] to ops[n_seq,n_ops]
def operators(s):
    #generate terms in the energy function
    n_seq,n_var = s.shape
    ops = np.zeros((n_seq,n_var+int(n_var*(n_var-1)/2.0)))

    jindex = 0
    for index in range(n_var):
        ops[:,jindex] = s[:,index]
        jindex +=1

    for index in range(n_var-1):
        for index1 in range(index+1,n_var):
            ops[:,jindex] = s[:,index]*s[:,index1]
            jindex +=1
            
    return ops
#=========================================================================================
def energy_ops(ops,w):
    return np.sum(ops*w[np.newaxis,:],axis=1)
#=========================================================================================
# 
def generate_seq(n_var,n_seq,n_sample=30,g=1.0):
    n_ops = n_var+int(n_var*(n_var-1)/2.0)
    #w_true = g*(np.random.rand(ops.shape[1])-0.5)/np.sqrt(float(n_var))
    w_true = np.random.normal(0.,g/np.sqrt(n_var),size=n_ops)
    
    samples = np.random.choice([1.0,-1.0],size=(n_seq*n_sample,n_var),replace=True)
    ops = operators(samples)

    #sample_energy = energy_ops(ops,w_true)
    sample_energy = ops.dot(w_true)

    p = np.exp(sample_energy)
    p /= np.sum(p)
    out_samples = np.random.choice(np.arange(n_seq*n_sample),size=n_seq,replace=True,p=p)
    
    return w_true,samples[out_samples]
#=========================================================================================
# find coupling w from sequences s
# input: ops[n_seq,n_ops]
# output: w[n_ops], E_av
def fit(ops,eps=0.1,max_iter=151,alpha=0.1,regular = 0.1):
    E_av = np.zeros(max_iter)
    n_ops = ops.shape[1]

    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5    
    for i in range(max_iter):
        #if eps_type == 'random':
        #    eps_scale = np.random.rand()/np.max([1.,np.max(np.abs(w))])
                                 
        #energies_w = energy_ops(ops,w)
        energies_w = ops.dot(w)

        energies_max = energies_w.max()  
        probs_w = np.exp((energies_w-energies_max)*(eps-1)) # to avoid a too lager value
        z_data = np.sum(probs_w)
        probs_w /= z_data
        ops_expect_w = np.sum(probs_w[:,np.newaxis]*ops,axis=0)

        #E_av[i] = energies_w.mean()    
        w += alpha*(ops_expect_w - w*eps - regular*w)
              
    return w#,-E_av[-1]

In [2]:
T = np.linspace(5.,0.001,50)
ram_arr = np.arange(2,0,-0.1)
L = 40
sample_size_arr = [5000]
ens = 15
eps = 0.8

In [ ]:
MSE_arr = np.zeros((len(T),ens,len(ram_arr)))

#---------------------------------------------------------------------------------------------------------------------
for en in tqdm(range(ens)):
    
    config_arr = np.zeros((L,1,50))

    path = r'E:\L40\저장\config\SK model\L40_{1}\\'.format(L,en+36)
    
    for r in range(50):
        config_arr_0 = np.load(path + 'config_%d.npy'%(r))
        config_arr = np.concatenate((config_arr,config_arr_0),axis = 1)

    randidx = np.random.choice(50000,5000,replace = False)
    config_arr = config_arr[:,1:,:]
    config_arr = config_arr[:,randidx,:]
    
    for r,ram in enumerate(ram_arr):
    #---------------------------------------------------------------------------------------------------------------------
        YOUR_DIRECTORY_NAME = r'C:\Users\user\Dropbox\test\각종 자료\Erasure Machine\저장\그림저장\inference\Regularization\L40\R{0}\\'.format(ram)
        try:
            if not(os.path.exists(YOUR_DIRECTORY_NAME)):
                os.makedirs(YOUR_DIRECTORY_NAME)
        except OSError as e:
            if OSError:
                print("Failed to create directory")
#---------------------------------------------------------------------------------------------------------------------
        w_mat = np.load(path + 'J.npy')
        iu1 = np.triu_indices(L,1)
        w_true = np.hstack((np.zeros(L),w_mat[iu1]))
    #---------------------------------------------------------------------------------------------------------------------

        for i,t in enumerate(T):
            seqs = config_arr[:,:,i].T
            ops = operators(seqs)
            w = fit(ops,eps=eps,max_iter=100,regular = ram)
            MSE_arr[i,en,r] = np.sum(w_true-w)**2 / np.sum(w_true**2)

            YOUR_DIRECTORY_NAME_en = r'C:\Users\user\Dropbox\test\각종 자료\Erasure Machine\저장\그림저장\inference\Regularization\L40\R{2}\en{0}\T{1}\\'.format(en+36,t,ram)

            try:
                if not(os.path.exists(YOUR_DIRECTORY_NAME_en)):
                    os.makedirs(YOUR_DIRECTORY_NAME_en)
            except OSError as e:
                if OSError:
                    print("Failed to create directory")

            np.save( YOUR_DIRECTORY_NAME_en + 'w_true.npy',w_true)
            np.save( YOUR_DIRECTORY_NAME_en + 'w_infered.npy',w)

            plt.plot([-1.,1.],[-1.,1.],'r--')
            plt.plot(w_true,w,'ko')
            plt.xlabel('Actual interactions')
            plt.ylabel('Inferred interactions')
            plt.savefig(YOUR_DIRECTORY_NAME_en + 'act_inf.svg')
            plt.savefig(YOUR_DIRECTORY_NAME_en + 'act_inf.png')
            plt.close()
            
np.save(YOUR_DIRECTORY_NAME + 'MSE.npy',MSE_arr)